<a href="https://colab.research.google.com/github/dinooooooi/dinooooi/blob/main/0322%EA%B8%88%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#7. 앙상블 학습과 랜덤 포레스트

일련의 예측기를 앙상블이라고 부르고 일련의 예측기로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻는 것을 앙상블 학습이라고 함. 앙상블 학습 알고리즘은 앙상블 방법.

랜덤 포레스트 : 결정트리의 앙상블


In [1]:
import warnings

warnings.filterwarnings('ignore')


# import package

import numpy as np

import os



#5장에서 소개한 moons dataset 불러오기

from sklearn.datasets import make_moons

from sklearn.model_selection import train_test_split

X,y = make_moons(n_samples=100, noise=0.15)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

##7.1 투표 기반 분류기
직접 투표: 각 분류기의 예측을 모아서 다수결로 클래스를 예측하는 것
각 분류기가 약한 학습기라도 앙상블은 강한 학습기 가능 - 큰 수의 법칙 때문

모든 분류기가 완벽히 독립적이고 오차에 상관관계가 없어야 함

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [3]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.95
RandomForestClassifier 1.0
SVC 1.0
VotingClassifier 1.0


투표기반 분류기가 다른 개별 분류기보다 성능이 높음\

간접 투표: 모든 분류기가 클래스의 확률을 예측할 수 있으면 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측

voting="soft"로 바꾸고 probability=True로 바꾸면 됨

##7.2 배깅과 페이스팅

배깅: 훈련 세트에서 중복을 허용해서 샘플링

페이스팅 : 중복을 허용하지 않고 샘플링

앙상블의 수집함수는 전형적 분류일 때 통계적 최빈값이고 희귀에 대해서는 평균 계산 - 편향은 비슷하나 분산은 줄어듦, 예측기는 모두 동시에 다른 cpu코어나 서버에서 병렬로 학습시킬 수 있음

###7.2.1 사이킷런의 배깅과 페이스팅

결정트리 분류기 500개의 앙상블을 훈련시키는 코드 / 중복 허용해 무작위로 선택된 100개의 샘플로 훈련

In [7]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf= BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1
)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

앙상블은 비슷한 편향에서 더 작은 분산 만듦

부트스트래핑은 각 예측기가 학습하는 서브셋에 다양성 증가시켜 배깅이 페이스팅보다 편향이 조금 더 높음. 다양성을 추가해 분산은 감소

###7.2.2 oob 평가
선택되지 않은 훈련 샘플의 나머지를 oob샘플이라고 부름

앙상블의 평가는 각 예측기의 oob 평가를 평균해 얻음

훈련이 끝난 후 자동으로 oob 평가를 수행

In [8]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1,oob_score=True
)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9125

In [10]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)
#비슷하게 정확도 얻음

1.0

In [11]:
bag_clf.oob_decision_function_

array([[0.91758242, 0.08241758],
       [0.        , 1.        ],
       [0.96703297, 0.03296703],
       [1.        , 0.        ],
       [0.01081081, 0.98918919],
       [0.3655914 , 0.6344086 ],
       [1.        , 0.        ],
       [0.88950276, 0.11049724],
       [0.02285714, 0.97714286],
       [0.38624339, 0.61375661],
       [0.        , 1.        ],
       [0.02185792, 0.97814208],
       [0.9695122 , 0.0304878 ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.14814815, 0.85185185],
       [0.14018692, 0.85981308],
       [1.        , 0.        ],
       [0.16111111, 0.83888889],
       [0.07065217, 0.92934783],
       [0.98477157, 0.01522843],
       [0.38095238, 0.61904762],
       [0.75977654, 0.24022346],
       [0.06024096, 0.93975904],
       [0.45026178, 0.54973822],
       [0.        , 1.        ],
       [0.9939759 , 0.0060241 ],
       [0.99450549, 0.00549451],
       [0.97860963, 0.02139037],
       [0.        , 1.        ],
       [0.

##7.3 랜덤 패치와 랜덤 서브스페이스

훈련 특성과 샘플을 모두 샘플링하는 것은 랜덤 패치 방식

훈련샘플은 모두 사용하고 특성은 샘플링하는 것을 랜덤 서브스페이스 방식

##7.4 랜덤 포레스트

500개 트리로 이뤄진 랜덤 포레스트 분류기를 여러 cPU 코어에서 훈련시키는 코드

In [12]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [15]:
#무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입.
#트리를 더 다양하게 만들고 편향을 손해보는 대신 분산을 낮춰 전체적으로 더 훌륭한 모델 만듦

In [14]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

###7.4.1 엑스트라 트리

익스티림 랜덤 트리 앙상블 : 극단적으로 무작위한 트리의 랜덤 포레스트

편향이 늘어나지만 대신 분산을 낮춤

엑스트라 트리는 특성마다 최적의 임곗값 찾지 않아 빠름

###7.4.2 특성 중요도

사이킷런은 훈련이 끝난 뒤 특성마다 자동으로 이 점수를 계산하고 중요도의 전체 합이 1이 되도록 결과값을 정규화함

iris data에서 각 특성의 중요도 출력

In [16]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
  print(name, score)

sepal length (cm) 0.10654786251474
sepal width (cm) 0.02283388367733349
petal length (cm) 0.45727170395130423
petal width (cm) 0.41334654985662234


##7,5 부스팅
부스팅은 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법

###7.5.1 에이다부스트
첫번째 분류기 훈련, 예측 만들기 : 알고리즘 기반 - 잘못 분류된 훈련 샘플의 가중치를 상대적으로 높임

두번째 분류기 훈련, 예측 만들기 : 업데이트된 가중치 사용, 반복

경사하강법과 비슷함


각 샘플의 가중치는 초기에 1/m으로 초기화됨, 첫번째 예측기가 학습되고 가중치가 적용된 에러율 r이 훈련세트에 대해 계산됨

예측기가 정확할수록 가중치가 높아짐 , 무작위는 가중치 0, 음수도 가능

가중치 업데이트됨. 잘못 분류된 샘플의 가중치 증가

마지막으로 새 예측기가 업데이트된 가중치를 사용해 훈련되고 전체 과정 반복

가중치 합이 가장 큰 클래스가 예측 결과가 됨

사이킷런은 SAMME라는 에이다부스트의 다중 클래스 버전을 사용, 예측값 대신 클래스 확률에 기반하며 일반적으로 성능이 더 좋음


200개의 아주 얕은 결정트리 기반으로 하는 에이다부스트 분류기 훈련

In [17]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5
)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

###7.5.2 그레이디언트 부스팅

: 앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가
이전 예측기가 만든 잔여 오차에 새로운 예측기 학습시킴

결정 트리를 기반 예측기로 사용하는 간단한 회귀 문제 : 그레이디언트 트리 부스팅, 그레이디언트 부스티드 회귀 트리라고 함

In [18]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [19]:
#첫번째 예측기에서 생긴 잔여 오차에 두번째 DecisionTreeRegressor 훈련

y2 = y-tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [20]:
#두번째 예측기가 만든 잔여 오차에 회귀 모델을 훈련

y3 = y2-tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [22]:
#새로운 샘플에 대한 예측을 만들려면 모든 트리의 예측을 더하면 됨

#y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [23]:
#트리가 앙상블에 추가될수록 앙상블의 예측이 점차 좋아짐

from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [25]:
#각 트리의 기여 정도를 조절하는 learning_rate 매개변수를 0.1처럼 낮게 설정하면 앙상블을 훈련세트에 학습시키기 위해 많은 트리가 필요하지만 일반적으로 예측의 성능은 좋아짐. = 축소

#간단하게 최적의 트리 수를 찾기 위해 조기종료 간단히 구현 : staged_predict()메서드

#120개의 트리로 GBRT 앙상블을 훈련시키고 최적의 트리 수를 찾기 위해 각 훈련 단계에서 검증 오차를 측정

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)+1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=120)

In [26]:
#warm_start=True로 설정하면 사이킷런이 fit()메서드가 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있도록 해줌

gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1,120):
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train, y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val, y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up=0
  else:
    error_going_up +=1
    if error_going_up==5:
      break # 조기 종료

In [27]:
# 확률적 그리이디언트 부스팅 : subsample 매개변수로 각 트리가 훈련할 때 사용할 훈련 샘플의 비율을 지정하는 기법

import xgboost
xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [29]:
xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.40614
[1]	validation_0-rmse:0.34795
[2]	validation_0-rmse:0.31837
[3]	validation_0-rmse:0.30655
[4]	validation_0-rmse:0.27702
[5]	validation_0-rmse:0.25511
[6]	validation_0-rmse:0.23999
[7]	validation_0-rmse:0.23013
[8]	validation_0-rmse:0.22269
[9]	validation_0-rmse:0.22191
[10]	validation_0-rmse:0.21750
[11]	validation_0-rmse:0.21735
[12]	validation_0-rmse:0.21602
[13]	validation_0-rmse:0.21509
[14]	validation_0-rmse:0.21436
[15]	validation_0-rmse:0.21383
[16]	validation_0-rmse:0.21337
[17]	validation_0-rmse:0.21358


##7.6 스태킹

사이킷런은 스태킹을 직접 지원하지 않음 ..

각각 다른 값을 예측하고 마지막 예측기(블렌더)가 이 예측을 입력으로 받아 최종 예측을 만듦

블렌더를 학습시키는 방법은 홀드아웃 세트를 사용하는 것